<a href="https://colab.research.google.com/github/junyamahira/ReinforcementLearning_Prac/blob/master/DQN_MountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# keras-rlのインストール

## git clone 
- これかpipのどちらかを実行すればよい (pipがおすすめ)
- keras-rlの中に入り、setup.pyを実行する

In [0]:
!git clone https://github.com/keras-rl/keras-rl.git

Cloning into 'keras-rl'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1710 (delta 3), reused 7 (delta 2), pack-reused 1698
Receiving objects: 100% (1710/1710), 1.38 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (1058/1058), done.


In [0]:
!ls keras-rl

assets		 examples	    mkdocs.yml	rl	   tests
CONTRIBUTING.md  ISSUE_TEMPLATE.md  pytest.ini	setup.cfg  utils
docs		 LICENSE	    README.md	setup.py


In [0]:
!cd keras-rl
!ls

keras-rl  sample_data


## pip

In [0]:
pip install keras-rl

# その他のimport

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import gym
from gym import wrappers
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


## visiual display

In [0]:
#?
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null

!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

#　

In [0]:
env = gym.make("MountainCar-v0")
env = wrappers.Monitor(env, './', force=True)
nb_actions = env.action_space.n

# モデル

In [0]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                48        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

In [0]:
memory = SequentialMemory(limit=50000, window_length=1)
# 方策
policy = EpsGreedyQPolicy(eps=0.001)
#　各パラメータをここで調整
dqn = DQNAgent(model=model, nb_actions=nb_actions,gamma=0.99, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
# 何設定してん？
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

history = dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)


Training for 50000 steps ...
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   173/50000: episode: 1, duration: 4.004s, episode steps: 173, steps per second: 43, episode reward: -173.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.139 [0.000, 2.000], mean observation: -0.223 [-1.200, 0.537], loss: 0.174112, mean_absolute_error: 0.700445, mean_q: -0.689521
   373/50000: episode: 2, duration: 3.053s, episode steps: 200, steps per second: 66, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.160 [0.000, 2.000], mean observation: -0.243 [-0.679, 0.025], loss: 0.004197, mean_absolute_error: 1.543724, mean_q: -2.184135
   573/50000: episode: 3, duration: 0.694s, episode steps: 200, steps per second: 288, episode reward: -200.000, mean reward: -1.000 [-1.000, -1.000], mean action: 1.725 [0.000, 2.000], mean observation: -0.213 [-0.545, 0.013], loss: 0.011902, mean_absolute_error: 2.517416, mean_q: -3.696577
   773/50000: episode: 4, duration: 0.623s, episode steps: 200, steps per second: 321, episode reward: -200.000, mean rewar

# 実行

## test  
ローカル上ではこれでアニメーションも表示される？  
colab上では実行してもしなくても同じ

In [0]:
dqn.test(env, nb_episodes=1, visualize=True)

## 確認
- colab上ではこっちを実行
- 2回以上実行するとエラーがでる。
原因、解決方法は調査中
同じような[issue](https://github.com/openai/gym/issues/430)

In [0]:
import gym
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline


for n_episode in range (3):
    #observation = env.reset()
    for t in range(100):
        plt.imshow(env.render(mode='rgb_array'))
        display.clear_output(wait=True)
        display.display(plt.gcf())

        action = env.action_space.sample() # アクションのランダム選択
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode {} finished after {} timesteps".format(n_episode+1,t+1))
            break

## ダウンロード  
これで学習の途中経過をダウンロードできる 

In [0]:
from google.colab import files
import glob

for file in glob.glob("openaigym.video.*.mp4"):
  files.download(file)

# 参考
[Keras Documentation　コールバック](https://keras.io/ja/callbacks/)  
[ColaboratoryでOpenAI gym](http://bcl.sci.yamaguchi-u.ac.jp/~jun/ja/blog/180828-openai-colaboratory)